# Konfigurasi Awal
Bagian konfigurasi awal untuk mendapatkan file input,
## Download from google cloud storage
Semua file input maupun output disimpan di cloud storage
### Nama file
file input: data/input-latest.csv
file output: data/output-latest.json
> Boleh juga jika modelnya ingin diexport
#### File input dikoleksi setiap hari dengan format input-timestamp.csv

In [3]:
import os
import glob
import pandas as pd
from datetime import datetime

In [11]:
import sys
!{sys.executable} -m pip install python-dotenv

In [4]:
from dotenv import load_dotenv

load_dotenv() 
os.environ["TFVERSION"] = "2.5"
os.environ["PYTHONVERSION"] = "3.7"

In [46]:
%%bash

mkdir -p data
gsutil -q -m cp gs://$BUCKET_NAME/data/input* data/

In [54]:
inputList = sorted(glob.glob('data/input-1*.csv'))
print(inputList)
isOk = glob.glob('data/input-aafsa.csv')
not inputList

['data/input-1622264519104.csv', 'data/input-1622350805158.csv', 'data/input-1622437205217.csv', 'data/input-1622523605201.csv']


False

In [191]:
def read_csv_add_date(path, isNeedParse=True):
    csvData = pd.read_csv(path)
    if isNeedParse:
        csvData["date"] = pd.to_datetime(
            datetime.fromtimestamp(
                int(path.split("-")[-1][:-4])//1000
            ))
    return csvData

In [201]:
def preprocess_merge():
    isMergedDatasetExist = glob.glob('data/input-preprocessed.csv')
    listOfDatasets = sorted(glob.glob('data/input-1*.csv'))
    if not isMergedDatasetExist:
        df_merge = pd.DataFrame()
        for i, dataset in enumerate(listOfDatasets[:-1]):
            df1 = read_csv_add_date(listOfDatasets[i])
            df2 = read_csv_add_date(listOfDatasets[i+1])
            df2["visitor"] = df2["visitor"].sub(df1["visitor"])
            if (i == 0):
                df_merge = df_merge.append(df1, ignore_index = True)
            df_merge = df_merge.append(df2, ignore_index = True)
            df_merge.head(10);
            
        df_merge = df_merge.sort_values(
            ["id", "date"], ascending = (True, True))
        df_merge["date"] = pd.to_datetime(
            df_merge.date).dt.strftime("%-d/%-m/%Y %-H:%M")
        
        df_merge.to_csv("data/input-preprocessed.csv", index=False)
#         display(df_merge.head())
        
    else:
        df_merge = read_csv_add_date('data/input-preprocessed.csv', isNeedParse=False)
        df_merge["date"] = pd.to_datetime(
            df_merge.date)
        
        df_to_merge = read_csv_add_date(
            listOfDatasets[-1]) # read previous cumulative dump
        df_to_sub = read_csv_add_date(
            listOfDatasets[-2]) # read previous 2 cumulative dump
        
        df_to_merge["visitor"] = df_to_merge["visitor"].sub(df_to_sub["visitor"]) # get non-cumulative visitor sum
        
        df_merge = df_merge.append(
            df_to_merge, ignore_index = True)
        df_merge = df_merge.sort_values(
            ["id", "date"], ascending = (True, True))
        
        df_merge["date"] = pd.to_datetime(
            df_merge.date).dt.strftime("%-d/%-m/%Y %-H:%M")
        df_merge.to_csv("data/input-preprocessed.csv", index=False)
#         display(df_merge.head())

In [202]:
preprocess_merge()

In [97]:
listOfDatasets = sorted(glob.glob('data/input-1*.csv'))
display(listOfDatasets)
display(listOfDatasets[:-1])


['data/input-1622264519104.csv',
 'data/input-1622350805158.csv',
 'data/input-1622437205217.csv',
 'data/input-1622523605201.csv']

['data/input-1622264519104.csv',
 'data/input-1622350805158.csv',
 'data/input-1622437205217.csv']

In [43]:
len(df_merge)
df_merge = df_merge.sort_values(["id", "date"], ascending = (True, False));

In [45]:
df_merge.head(10)

,id,visitor,interest,contact,name,place_id,date
0,0,1,0,6281329167890,RUMAH PRODUKSI BUMBU UMAY,ChIJSbe7edeRei4RNsJWmGkQBPI,29/5/2021 5:01
12,0,1,0,6281329167890,RUMAH PRODUKSI BUMBU UMAY,ChIJSbe7edeRei4RNsJWmGkQBPI,1/6/2021 5:00
1,1,1,0,6281542508821,Sirup Jahe Dewa,ChIJu1cJaGWPei4R2PH1rwpq4gg,29/5/2021 5:01
13,1,3,0,6281542508821,Sirup Jahe Dewa,ChIJu1cJaGWPei4R2PH1rwpq4gg,1/6/2021 5:00
2,2,1,0,6281329167890,Sambal Mbak Ririn,ChIJUUAuMWSFei4R0SORqr3pquU,29/5/2021 5:01
14,2,2,0,6281329167890,Sambal Mbak Ririn,ChIJUUAuMWSFei4R0SORqr3pquU,1/6/2021 5:00
3,3,1,0,6285796936649,Sambal Pawon Soendari Magelang,ChIJzZ8nRZCFei4RXmg0wdxJ23s,29/5/2021 5:01
15,3,3,0,6285796936649,Sambal Pawon Soendari Magelang,ChIJzZ8nRZCFei4RXmg0wdxJ23s,1/6/2021 5:00
4,4,1,0,6285868828975,Getuk Eco,ChIJe6ObOVyPei4Rr9PqdGqxtkE,29/5/2021 5:01
16,4,3,0,6285868828975,Getuk Eco,ChIJe6ObOVyPei4Rr9PqdGqxtkE,1/6/2021 5:00


In [24]:
df_merge.to_csv("data/input-raw.csv", index=False)

### Kode Machine Learning
> Masukan Kode ML dari cell ini

### Upload file output
File output diupload ke cloud storage

#### Format file output: 
`output-latest.json` dan `output-[timestamp].json`, boleh juga kalau mau upload bentuk lainnya (misal HDF5 nya dll.)
> Pastikan output **disimpan ke direktori data**

In [5]:
%%bash

gsutil cp data/output* gs://$BUCKET_NAME/data/

Copying file://data/output-latest.json [Content-Type=application/json]...
/ [1 files][  2.7 KiB/  2.7 KiB]                                                
Operation completed over 1 objects/2.7 KiB.                                      
